### Data Cleaning
1. The data retrieved from CDC were in multiple files that are year dependent. The files will need to be combined
2. Once the files are combined then all unnecessary attributes will be removed. 


## 1. Table Combine

In [72]:
import os
import pandas as pd

def combine_csv_files(input_folder, output_file):
    # Check if the output file already exists
    if os.path.exists(output_file):
        print(f"The output file '{output_file}' already exists. No changes were made.")
        return
    
    # List to hold dataframes
    dataframes = []
    
    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)
    
    # Concatenate all dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Save the combined dataframe to the specified output file
    combined_df.to_csv(output_file, index=False)
    print(f"All CSV files from {input_folder} have been combined and saved to {output_file}.")

# Create Combined file
input_folder = r"C:\Users\laura\OneDrive\Documents\capstone-dooley\data\antibiotic usage\prescriptions"
output_file = 'combined_prescription_rate.csv'
input_folder2 = r'C:\Users\laura\OneDrive\Documents\capstone-dooley\data\antibiotic usage\saar'
output_file2 = 'combined_saar.csv'
input_folder3 = r'C:\Users\laura\OneDrive\Documents\capstone-dooley\data\resistance'
output_file3 = 'combined_resistance.csv'

combine_csv_files(input_folder, output_file)
combine_csv_files(input_folder2, output_file2)
combine_csv_files(input_folder3, output_file3)

All CSV files from C:\Users\laura\OneDrive\Documents\capstone-dooley\data\antibiotic usage\prescriptions have been combined and saved to combined_prescription_rate.csv.
All CSV files from C:\Users\laura\OneDrive\Documents\capstone-dooley\data\antibiotic usage\saar have been combined and saved to combined_saar.csv.


C:\Users\laura\AppData\Local\Temp\ipykernel_7488\3393467354.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


All CSV files from C:\Users\laura\OneDrive\Documents\capstone-dooley\data\resistance have been combined and saved to combined_resistance.csv.


## summary of the new files 

In [73]:
def summarize_csv(file):
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Get the number of rows and columns
    num_rows, num_columns = df.shape
    
    # Get column names and data types
    column_info = df.dtypes

     
    # Get summary statistics for numerical columns
    summary_stats = df.describe()
    
    # Print the summary
    print(f"Number of rows: {num_rows}")
    print(f"Number of columns: {num_columns}")
    print("\nColumn Information:")
    print(column_info)
    print("\nSummary Statistics for Numerical Columns:")
    print(summary_stats)

# Example usage
file = 'combined_prescription_rate.csv'
file2 = 'combined_resistance.csv'
file3 = 'combined_saar.csv'

summarize_csv(file)

summarize_csv(file2)

summarize_csv(file3)

Number of rows: 310
Number of columns: 6

Column Information:
ID State                                int64
State                                  object
Year                                  float64
Prescriptions per 1,000 Enrollees     float64
Event Year                            float64
Prescriptions per 1,000 Population    float64
dtype: object

Summary Statistics for Numerical Columns:
         ID State         Year  Prescriptions per 1,000 Enrollees  \
count  310.000000   102.000000                         102.000000   
mean    28.587097  2017.500000                        1422.558824   
std     15.940337     0.502469                         218.322478   
min      0.000000  2017.000000                         996.000000   
25%     16.000000  2017.000000                        1235.500000   
50%     29.000000  2017.500000                        1408.000000   
75%     41.750000  2018.000000                        1575.000000   
max     56.000000  2018.000000                       

## data adjustments 
1. saar.csv  - needs to switch data type, remove zeros, and add the SAAR value (observed days/predicted days) in a new column 


In [93]:
# Load the CSV file into a DataFrame

df = pd.read_csv(file3)

# Ensure there are no zero values in 'Predicted Antimicrobial Days'
if (df['Predicted Antimicrobial Days'] == 0).any():
    print("Warning: Found zero values in 'Predicted Antimicrobial Days'. These will be replaced with NaN.")
    df['Predicted Antimicrobial Days'].replace(0, pd.NA, inplace=True)

# Add a new column named 'SAAR' while handling division safely
df['SAAR'] = df['Observed Antimicrobial Days'] / df['Predicted Antimicrobial Days']

# Convert the 'SAAR' column to int64 (dropping NaN before conversion)
df['SAAR'] = df['SAAR'].round(4)

output_file = 'updated_saar'
df.to_csv(output_file, index=False)

print(f"SAAR column added, converted to int64, and saved to {output_file}")



SAAR column added, converted to int64, and saved to updated_saar


2. resistance.csv - The agecat contains all/adult/peds. All needs to be removed since it is the combination of adult and peds.  

In [94]:
df = pd.read_csv('combined_resistance.csv')

# Remove rows where the 'agecat' column is "All"
df = df[df['agecat'] != "All"]

# Save the updated DataFrame to a new file
output_file_path = 'updated_resistance.csv'
df.to_csv(output_file_path, index=False)

print(f"Rows with 'All' in the 'agecat' column removed and saved to {output_file_path}")

Rows with 'All' in the 'agecat' column removed and saved to updated_resistance.csv


In [78]:
# Read the data from the CSV file
df = pd.read_csv("combined_resistance.csv")

# Convert 'pctNonSuscep' to numeric, coercing non-numeric entries to NaN
df['pctNonSuscep'] = pd.to_numeric(df['pctNonSuscep'], errors='coerce')

# Drop rows with NaN values in 'pctNonSuscep'
df.dropna(subset=['pctNonSuscep'], inplace=True)

#  convert it to int64
df['pctNonSuscep'] = df['pctNonSuscep'].astype('int64')


In [80]:

# Dictionary mapping state names to abbreviations
state_to_abbreviation = {
    "ALABAMA": "AL", "ALASKA": "AK", "ARIZONA": "AZ", "ARKANSAS": "AR",
    "CALIFORNIA": "CA", "COLORADO": "CO", "CONNECTICUT": "CT", "DELAWARE": "DE",
    "FLORIDA": "FL", "GEORGIA": "GA", "HAWAII": "HI", "IDAHO": "ID",
    "ILLINOIS": "IL", "INDIANA": "IN", "IOWA": "IA", "KANSAS": "KS",
    "KENTUCKY": "KY", "LOUISIANA": "LA", "MAINE": "ME", "MARYLAND": "MD",
    "MASSACHUSETTS": "MA", "MICHIGAN": "MI", "MINNESOTA": "MN", "MISSISSIPPI": "MS",
    "MISSOURI": "MO", "MONTANA": "MT", "NEBRASKA": "NE", "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH", "NEW JERSEY": "NJ", "NEW MEXICO": "NM", "NEW YORK": "NY",
    "NORTH CAROLINA": "NC", "NORTH DAKOTA": "ND", "OHIO": "OH", "OKLAHOMA": "OK",
    "OREGON": "OR", "PENNSYLVANIA": "PA", "RHODE ISLAND": "RI", "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD", "TENNESSEE": "TN", "TEXAS": "TX", "UTAH": "UT",
    "VERMONT": "VT", "VIRGINIA": "VA", "WASHINGTON": "WA", "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI", "WYOMING": "WY"
}

# Load your dataset
file_path = r'C:\Users\laura\OneDrive\Documents\capstone-dooley\data\livestock_sales.csv'
df = pd.read_csv(file_path)

# Function to convert state names to abbreviations
def convert_to_abbreviation(state):
    return state_to_abbreviation.get(state.upper(), state)  # Default to state if no match found

# Apply the function to the State column
df['state'] = df['state'].apply(convert_to_abbreviation)

# Clean up the "Value" column to remove commas and convert to numeric
df['Value'] = df['Value'].str.replace(',', '', regex=True)  # Remove commas
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')  # Convert to numeric

# Drop rows with NaN values in 'Value'
df.dropna(subset=['Value'], inplace=True)

# Save the updated dataset
output_path = 'updated_livestock_sales_file.csv'
df.to_csv(output_path, index=False)

print(f"Updated file saved to {output_path}")




Updated file saved to updated_livestock_sales_file.csv


switched states to abbrevation on combined_saar.csv file

In [83]:
# Dictionary mapping state names to abbreviations
state_to_abbreviation = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY",
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY"
}

# Load your dataset
df = pd.read_csv("combined_saar.csv")

# Function to convert state names to abbreviations
def convert_to_abbreviation(state):
    return state_to_abbreviation.get(state, state)  # Default to state if no match found

# Apply the function to the state column
df['state'] = df['state'].apply(convert_to_abbreviation)

# Save the updated dataset
df.to_csv("updated_saar_file.csv", index=False)

# Check the first few rows to verify
print(df.head())


  SAAR Population Agent  Event Year state  ID State  \
0           Adult   All        2021    AL         1   
1           Adult   All        2021    AK         2   
2           Adult   All        2021    AZ         4   
3           Adult   All        2021    AR         5   
4           Adult   All        2021    CA         6   

   No. of facilities reporting SAARs  \
0                                 20   
1                                  5   
2                                 11   
3                                 24   
4                                109   

   No. of facilities eligible to report SAARs  \
0                                          91   
1                                          16   
2                                          78   
3                                          66   
4                                         362   

  Percentage of eligible facilities reporting  Observed Antimicrobial Days  \
0                                      21.90%          

In [95]:
def calculate_average_per_state(file_name, column_name, state_column):
    # Load the dataset
    df = pd.read_csv(file_name)
    
    # Ensure the specified column is numeric
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    
    # Handle NaN values by filling them with 0
    df[column_name] = df[column_name].fillna(0)
    
    # Calculate the average of the specified column per state
    average_per_state = df.groupby(state_column)[column_name].mean().reset_index()
    
    # Rename columns for clarity
    average_per_state.columns = [state_column, f'avg_{column_name}']
    
    # Check if avg_state.csv exists
    try:
        existing_df = pd.read_csv("avg_state.csv")
        # Ensure the state column names match
        if state_column != existing_df.columns[0]:
            existing_df.rename(columns={existing_df.columns[0]: state_column}, inplace=True)
        # Merge the new average data with the existing data
        result_df = pd.merge(existing_df, average_per_state, on=state_column, how='outer')
    except FileNotFoundError:
        # If the file does not exist, use the new average data as the result
        result_df = average_per_state
    
    # Save the result to avg_state.csv
    result_df.to_csv("avg_state.csv", index=False)
    
    return result_df

# Example usage
file_name = "updated_saar_file.csv"
column_name = "SAAR"
state_column = "state"
file_name2 = "updated_resistance.csv"
column_name2 = "pctNonSuscep"
file_name3 = "updated_livestock_sales_file.csv"
column_name3 = "Value"


# Calculate averages and save to avg_state.csv
calculate_average_per_state(file_name, column_name, state_column)
calculate_average_per_state(file_name2, column_name2, state_column)
calculate_average_per_state(file_name3, column_name3, state_column)



,state,avg_SAAR_x,avg_SAAR_y,avg_pctNonSuscep_x,avg_Value_x,avg_SAAR,avg_pctNonSuscep_y,avg_Value_y
0,AK,0.000000,0.000000,0.439368,3.209917e+07,0.000000,0.329569,3.209917e+07
1,AL,0.353875,0.353875,7.421897,4.211669e+09,0.353875,5.606552,4.211669e+09
2,AR,0.378725,0.378725,5.125718,5.141045e+09,0.378725,3.747500,5.141045e+09
3,AZ,0.254575,0.254575,6.923161,1.399922e+09,0.254575,5.201681,1.399922e+09
4,CA,0.982988,0.982988,14.065431,1.062359e+10,0.982988,11.721681,1.062359e+10
5,CO,0.483175,0.483175,4.649109,4.598994e+09,0.483175,3.507069,4.598994e+09
6,CT,0.331825,0.331825,4.389023,1.617532e+08,0.331825,3.231034,1.617532e+08
7,DE,0.000000,0.000000,1.619454,9.236823e+08,0.000000,1.164397,9.236823e+08
8,FL,0.994413,0.994413,12.014282,1.587410e+09,0.994413,9.443793,1.587410e+09
9,GA,0.921362,0.921362,9.830661,5.381134e+09,0.921362,7.558233,5.381134e+09


adding latitude and longitude to the average_saar_per_state.csv

In [29]:
df = pd.read_csv("average_saar_per_state.csv")

# Add latitude and longitude columns based on the dictionary
df['latitude'] = df['state'].map(lambda x: state_coords[x][0] if x in state_coords else None)
df['longitude'] = df['state'].map(lambda x: state_coords[x][1] if x in state_coords else None)

# Save the updated DataFrame to a new CSV file
df.to_csv("average_saar_per_state_with_coords.csv", index=False)

# Print the updated DataFrame
print(df)

   state  avg_saar   latitude   longitude
0     AK       NaN  61.370716 -152.404419
1     AL  0.943667  32.806671  -86.791130
2     AR  1.009933  34.969704  -92.373123
3     AZ  1.018300  33.729759 -111.431221
4     CA  0.982988  36.116203 -119.681564
5     CO  0.773080  39.059811 -105.311104
6     CT  0.884867  41.597782  -72.755371
7     DE       NaN  38.989590  -75.505987
8     FL  0.994413  27.766279  -81.686783
9     GA  0.921362  33.040619  -83.643074
10    HI       NaN  21.094318 -157.498337
11    IA  0.895733  42.011539  -93.210526
12    ID  0.810450  44.240459 -114.478828
13    IL  0.924314  40.349457  -88.986137
14    IN  0.851263  39.849426  -86.258278
15    KS  0.914933  38.526600  -96.726486
16    KY  0.970880  37.668140  -84.670067
17    LA  1.164160  31.169546  -91.867805
18    MA  0.854800  42.230171  -71.530106
19    MD  0.957350  39.063946  -76.802101
20    ME  0.901067  44.693947  -69.381927
21    MI  0.857240  43.326618  -84.536095
22    MN  1.020020  45.694454  -93